<h1>Canada Road Vehicles Collision Predictive Study </h1>

<h2>To better understand this project scope, please, view the Introduction to the problem notebook.</h2>

In [65]:
print("importing the introduction to the problem notebook:")
%run "Introduction to the problem.ipynb"


importing the introduction to the problem notebook:
Importing Libraries...
Libraries imported!
car_accident_df imported successfully!
The car_accident_df shape is: (5860405, 22)


In [66]:

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>The raw code for this IPython notebook is by default hidden for easier reading.
To toggle on/off the raw code, click <a href="javascript:code_toggle()">here</a>.''')

<p>Review on our dataframe information:</p>

In [67]:
car_accident_df.head()

,C_YEAR,C_MNTH,C_WDAY,C_HOUR,C_SEV,C_VEHS,C_CONF,C_RCFG,C_WTHR,C_RSUR,...,V_ID,V_TYPE,V_YEAR,P_ID,P_SEX,P_AGE,P_PSN,P_ISEV,P_SAFE,P_USER
0,1999,1.0,1.0,20.0,2,2.0,34.0,NaN,1.0,5.0,...,1.0,6.0,1990.0,1.0,M,41.0,11.0,1.0,NaN,1.0
1,1999,1.0,1.0,20.0,2,2.0,34.0,NaN,1.0,5.0,...,2.0,1.0,1987.0,1.0,M,19.0,11.0,1.0,NaN,1.0
2,1999,1.0,1.0,20.0,2,2.0,34.0,NaN,1.0,5.0,...,2.0,1.0,1987.0,2.0,F,20.0,13.0,2.0,2.0,2.0
3,1999,1.0,1.0,8.0,2,1.0,1.0,NaN,5.0,3.0,...,1.0,1.0,1986.0,1.0,M,46.0,11.0,1.0,NaN,1.0
4,1999,1.0,1.0,8.0,2,1.0,1.0,NaN,5.0,3.0,...,99.0,NaN,NaN,1.0,M,5.0,99.0,2.0,NaN,3.0


In [68]:
print("car_accident_df shape is:", car_accident_df.shape)

car_accident_df shape is: (5860405, 22)


In [69]:
print("Describe the df:")
car_accident_df.describe()

Describe the df:


,C_YEAR,C_MNTH,C_WDAY,C_HOUR,C_SEV,C_VEHS,C_CONF,C_RCFG,C_WTHR,C_RSUR,...,C_TRAF,V_ID,V_TYPE,V_YEAR,P_ID,P_AGE,P_PSN,P_ISEV,P_SAFE,P_USER
count,5.860405e+06,5.860020e+06,5.859082e+06,5.800996e+06,5.860405e+06,5.859861e+06,5.396406e+06,5.211459e+06,5.757417e+06,5.611737e+06,...,5.554904e+06,5.859972e+06,5.577294e+06,5.276027e+06,5.849413e+06,5.465249e+06,5.762728e+06,5.488993e+06,4.626121e+06,5.684819e+06
mean,2.005999e+03,6.714063e+00,4.010584e+00,1.369879e+01,1.983170e+00,2.045766e+00,2.269863e+01,1.717369e+00,1.605963e+00,1.582910e+00,...,1.080344e+01,5.194857e+00,2.250568e+00,1.998515e+03,1.458028e+00,3.639899e+01,1.649438e+01,1.574630e+00,2.496463e+00,1.502979e+00
std,4.569887e+00,3.433410e+00,1.932659e+00,5.197796e+00,1.286357e-01,1.268792e+00,1.155906e+01,7.804579e-01,1.115974e+00,1.110135e+00,...,8.075934e+00,1.851911e+01,3.603703e+00,7.126688e+00,1.204447e+00,1.836923e+01,1.817446e+01,5.090521e-01,2.259618e+00,8.461417e-01
min,1.999000e+03,1.000000e+00,1.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,...,1.000000e+00,1.000000e+00,1.000000e+00,1.901000e+03,1.000000e+00,1.000000e+00,1.100000e+01,1.000000e+00,1.000000e+00,1.000000e+00
25%,2.002000e+03,4.000000e+00,2.000000e+00,1.000000e+01,2.000000e+00,2.000000e+00,2.100000e+01,1.000000e+00,1.000000e+00,1.000000e+00,...,1.000000e+00,1.000000e+00,1.000000e+00,1.994000e+03,1.000000e+00,2.200000e+01,1.100000e+01,1.000000e+00,2.000000e+00,1.000000e+00
50%,2.006000e+03,7.000000e+00,4.000000e+00,1.400000e+01,2.000000e+00,2.000000e+00,2.100000e+01,2.000000e+00,1.000000e+00,1.000000e+00,...,1.800000e+01,1.000000e+00,1.000000e+00,1.999000e+03,1.000000e+00,3.400000e+01,1.100000e+01,2.000000e+00,2.000000e+00,1.000000e+00
75%,2.010000e+03,1.000000e+01,6.000000e+00,1.700000e+01,2.000000e+00,2.000000e+00,3.500000e+01,2.000000e+00,2.000000e+00,2.000000e+00,...,1.800000e+01,2.000000e+00,1.000000e+00,2.004000e+03,2.000000e+00,4.900000e+01,1.300000e+01,2.000000e+00,2.000000e+00,2.000000e+00
max,2.014000e+03,1.200000e+01,7.000000e+00,2.300000e+01,2.000000e+00,7.700000e+01,4.100000e+01,1.000000e+01,7.000000e+00,9.000000e+00,...,1.800000e+01,9.900000e+01,2.300000e+01,2.015000e+03,9.900000e+01,9.900000e+01,9.900000e+01,3.000000e+00,1.300000e+01,5.000000e+00


<h2>Data pre-processing and selection:</h2>

<p>The data for this project was already pre-processed, with all the labels and definitions in conformation to the appended archive "attributes_legend.pdf".</p>

<p>To verify how much non numerical values influence in the dataframe, we will create a function that evaluates the NAn vlaues:</p>

In [70]:
def access_NA(data):
    """
    Returns a pandas dataframe denoting the total number of NA values and the percentage of NA values in each column.
    The column names are noted on the index.
    
    Parameters
    ----------
    data: dataframe
    """
    # pandas series denoting features and the sum of their null values
    null_sum = data.isnull().sum()# instantiate columns for missing data
    total = null_sum.sort_values(ascending=False)
    percent = ( ((null_sum / len(data.index))*100).round(2) ).sort_values(ascending=False)
    
    # concatenate along the columns to create the complete dataframe
    df_NA = pd.concat([total, percent], axis=1, keys=['Number of NA', 'Percent NA'])
    
    # drop rows that don't have any missing data; omit if you want to keep all rows
    df_NA = df_NA[ (df_NA.T != 0).any() ]
    
    return df_NA

In [71]:
access_NA(car_accident_df)

,Number of NA,Percent NA
P_SAFE,1234284,21.06
C_RCFG,648946,11.07
V_YEAR,584378,9.97
C_CONF,463999,7.92
C_RALN,463312,7.91
P_AGE,395156,6.74
P_ISEV,371412,6.34
C_TRAF,305501,5.21
V_TYPE,283111,4.83
P_SEX,249740,4.26


<p>Looking into those values, we see that we can't just use pandas "df.drop_na()" function, since it would drop a big percentage of the data. Therefore, we will use the drop_na function to the velues that represents low percentages of the total (up to 0,2 percent), and substitute the other values with pandas "fill_na()", using the mean values for the columns. For the specific case of the column "P_SEX", qhich is person sex, we will use the dropna approach.</p>

In [72]:
# Impute with mean in the whole df
columns_to_be_changed = ["P_SAFE", "C_RCFG", "V_YEAR", "C_CONF", "C_RALN", "P_AGE", "P_ISEV", "C_TRAF", "V_TYPE", "C_RSUR", "P_USER", "C_WTHR", "P_PSN", "C_HOUR"]
for i in columns_to_be_changed:
 car_accident_df[i] = car_accident_df[i].fillna( int(car_accident_df[i].mean()) )


In [73]:
access_NA(car_accident_df)

,Number of NA,Percent NA
P_SEX,249740,4.26
P_ID,10992,0.19
C_WDAY,1323,0.02
C_VEHS,544,0.01
V_ID,433,0.01
C_MNTH,385,0.01


In [83]:
#car_accident_df["P_SEX"].value_counts(normalize=True)

In [84]:
#car_accident_df["P_SEX"].value_counts()

<p>Now we drop the other values and we will have our df prepared.</p>

In [86]:
car_accident_df =  car_accident_df.dropna()




In [88]:
#Make sure there isn't any other N/A values:
access_NA(car_accident_df)

,Number of NA,Percent NA


<p>Now, the data is ready to be put to test.</p>